# Debugging

**Table of contents**<a id='toc0_'></a>    
- 1. [Example](#toc1_)    
- 2. [Numpy warnings](#toc2_)    
- 3. [Scope bugs](#toc3_)    
- 4. [Index bugs](#toc4_)    
- 5. [Debugger](#toc5_)    
  - 5.1. [In notebooks](#toc5_1_)    
  - 5.2. [In modules](#toc5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import numpy as np

**Video:** [Why is a programming error called a bug?](https://www.youtube.com/watch?v=rhFSG-VyR_E)

**General advice:**

1. Code is always partly a **black box**
2. **Print and plot results** to convince yourself (and others) that your results are sensible.
3. Errors are typically something **very very simple**, look after that.
4. If Python raises an error first try to **locate the line** where the error occurs.
5. Your code can often run, but give you **unexpected behavior**.

**Most of the time spend programming is debugging!!**

**Assertions:** You can enforce assertations on e.g. numeric values

In [2]:
x = -2
y = x**2
assert y > 0, f'x = {x}, y = {y}'

**Task:** Make the above assertion fail.

**Exceptions:** When code fails, it generates (*raises*) an exception 

In [3]:
x = 0.0
try:
    if x < 0.0:
        raise ValueError('x cannot be negative')
    else:
        print('successful')
except Exception as e:
    print(e)

successful


**Task:** Make it raise the exception and print *x cannot be negative*.

## 1. <a id='toc1_'></a>[Example](#toc0_)


Consider the following code:

In [4]:
a = 0.8
xlist = [-1,2,3]

def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        y += z
    return y

y = myfun(xlist,a)
print(y)

(3.340308817497993+0.5877852522924732j)


**Problem:** Our result is a complex number. We did not expect that. Why does this problem arise?

**Find the error with print:**

In [5]:
def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        print(f'x = {x} -> {z}') # temp
        y += z
    return y

myfun(xlist,a)

x = -1 -> (-0.8090169943749473+0.5877852522924732j)
x = 2 -> 1.7411011265922482
x = 3 -> 2.4082246852806923


(3.340308817497993+0.5877852522924732j)

**Solution with an assert:**

In [6]:
def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        assert np.isreal(z), f'z is not real for x = {x}, but {z}'
        y += z
    return y
    
try:
    myfun(xlist,a)
except Exception as e:
    print(e)

z is not real for x = -1, but (-0.8090169943749473+0.5877852522924732j)


**Solution with if and raise exception:**

In [7]:
def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        if not np.isreal(z):
            print(f'z is not real for x = {x}, but {z}')
            raise ValueError('negative input number') # an exception will be raised here  
        y += z
    return y

try:
    myfun(xlist,a)
except Exception as e:
    # we'll end up down here because the exception was raised. 
    print(e)

z is not real for x = -1, but (-0.8090169943749473+0.5877852522924732j)
negative input number


**Note:** You could also decide that the function should return e.g. **nan** when experiencing a complex number.

In [8]:
def myfun(xlist,a):
    y = 0
    for x in xlist:
        z = x**a
        if not np.isreal(z):
            return np.nan
        y += z
    return y

myfun(xlist,a)

nan

## 2. <a id='toc2_'></a>[Numpy warnings](#toc0_)

Here we see an example of a *RuntimeWarning*.

In [9]:
xlist = [-1,2,3]
def f(xlist):
    y = np.empty(len(xlist))
    for i,x in enumerate(xlist):
        y[i] = np.log(x)
    return y

f(xlist)

C:\Users\gmf123\AppData\Local\Temp\ipykernel_11612\3017935525.py:5: RuntimeWarning: invalid value encountered in log
  y[i] = np.log(x)


array([       nan, 0.69314718, 1.09861229])

You can **ignore all warnings**:

In [10]:
def f(xlist):
    
    y = np.empty(len(xlist))
    for i,x in enumerate(xlist):
        with np.errstate(all='ignore'):
            y[i] = np.log(x)
    return y

f(xlist)

array([       nan, 0.69314718, 1.09861229])

**Better:** Decide what the code should do.

In [11]:
def f(xlist):
    
    y = np.empty(len(xlist))
    for i,x in enumerate(xlist):
        if x <= 0:
            y[i] = -np.inf
        else:
            y[i] = np.log(x)
    return y

f(xlist)

array([      -inf, 0.69314718, 1.09861229])

## 3. <a id='toc3_'></a>[Scope bugs](#toc0_)

Global variables are dangerous:

In [12]:
# a. define a function to multiply a variable with 5
a = 5
def f(x):
    return a*x

# many lines of code
# many lines of code
# many lines of code

# z. setup the input and call f
y = np.array([3,3])
a = np.mean(y)
b = np.mean(f(y))

print(b)

9.0


**Question:** What is the error?

**Conclusion:** 

1. Never use global variables, they can give poisonous **side effects**.
2. Use a *positional argument* or a *keyword argument* instead. 

## 4. <a id='toc4_'></a>[Index bugs](#toc0_)

In [13]:
# a. setup
N = 10
x = np.linspace(1.3,8.2,N)
y = 9.2

# b. count all entries in x below y
i = 0
try:
    while x[i] < y:
        i += 1
except Exception as e:
    print(f'error found: {e}')

error found: index 10 is out of bounds for axis 0 with size 10


**Task:** Solve the problem.

## 5. <a id='toc5_'></a>[Debugger](#toc0_)

### 5.1. <a id='toc5_1_'></a>[In notebooks](#toc0_)

Consider this example:

In [15]:
def f(x):
    y = x-5
    z = x
    return np.log(y*z)

In [16]:
x = 4
q = f(x)
print(q)

nan


C:\Users\gmf123\AppData\Local\Temp\ipykernel_11612\529491813.py:4: RuntimeWarning: invalid value encountered in log
  return np.log(y*z)


**Task:** Let us analyze why the problem arises with the *debugger*.

1. Go to first line of `f(x)` two cells above. Press `F9` to create a *breakpoint*
2. Go to the cell just above this cell. Press `Ctrl+Shift+Alt+Enter`
3. Press `F10` to `Step Over`. Notice the value of y?
4. Exit with `Shift+F5`

**Extra:** Place the breakpoint at `x = 4`. Try out `F11` to `Step Into` the function `f(x)` and `Shift+F11` to `Step Out`

**More details:** See [here](https://code.visualstudio.com/docs/datascience/jupyter-notebooks#_debug-cell).

### 5.2. <a id='toc5_2_'></a>[In modules](#toc0_)

We can also use the debugger in this case.

In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
import mymodule

In [19]:
x = 4
q = mymodule.g(x)
print(q)

nan


c:\Users\gmf123\Dropbox\Repositories\course\IntroProg-lectures\1 - Debugging\mymodule.py:6: RuntimeWarning: invalid value encountered in log
  return np.log(y*z)
